In [86]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import json
from config.config_jupyter import set_wd
set_wd()
import json
from src.links import scrape_links_bp, Scraper
from src.wrangling import clean_bp_data_frame
from src.plots import plot_segun_codigo

c:\Users\mbasualdo\Documents\Analysis bp


In [87]:
scraper_instance = Scraper()
links_xls = scrape_links_bp(scraper_instance)

In [88]:
links_xls[0]

'https://www.indec.gob.ar/ftp/cuadros/economia/cin_III_2023.xls'

In [92]:
def _modify_bdp_code_anaf_enep(df:pd.DataFrame):
    def _assign_asset_liability(description):
        if description == 'Activos':
            return 'activo'
        elif description == 'Pasivos':
            return 'pasivo'
        else:
            return np.nan

    def _modify_bdp_code(description, bdp_code):
        if description == 'Activos':
            return 'A'
        elif description == 'Pasivos':
            return 'P'
        else:
            return bdp_code

    def _find_first_asset_or_liability(row, df):
        index_current = row.name
        while index_current >= 0:
            if 'activo_pasivo' in df.columns:
                value_previous = df.loc[index_current, 'activo_pasivo']
                if pd.notnull(value_previous):
                    return value_previous
            index_current -= 1
        return np.nan

    def modify_bdp_code_with_asset_liability(asset_liability, first_asset_or_liability, bdp_code):
        if pd.isna(asset_liability):
            if first_asset_or_liability == 'activo':
                return 'A.' + str(bdp_code)
            elif first_asset_or_liability == 'pasivo':
                return 'P.' + str(bdp_code)
        return bdp_code

    # Aplicamos las funciones para modificar 'Codigo BDP' y añadir 'activo_pasivo'
    df['Codigo BDP'] = df.apply(lambda x: _modify_bdp_code(x['Descripción'], x['Codigo BDP']), axis=1)
    df['activo_pasivo'] = df['Descripción'].apply(_assign_asset_liability)

    # Encontramos el primer activo o pasivo y lo aplicamos para modificar 'Codigo BDP'
    df['first_a_o_p'] = df.apply(lambda x: _find_first_asset_or_liability(x, df), axis=1)
    df['Codigo BDP'] = df.apply(lambda x: modify_bdp_code_with_asset_liability(x['activo_pasivo'], x['first_a_o_p'], x['Codigo BDP']), axis=1)

    df = df.drop(["activo_pasivo","first_a_o_p"], axis=1)
    return df

In [100]:
df = pd.read_excel(links_xls[0], sheet_name = "Cuadro 22", header=4, skipfooter=6)
df = df.rename({df.columns[1]:"Codigo BDP", df.columns[2]: "Descripción", df.columns[0]:"SDMX"}, axis = 1)
df = df.drop(df.columns[3:13], axis=1) #Hay años sin trimestres
df = _modify_bdp_code_anaf_enep(df)
df = (
    df.filter(regex='^(?!Total).*')
    .iloc[1:]
    .drop(["SDMX", "Descripción"], axis=1)
    .dropna(axis=0, thresh=2)
    .dropna(axis=1, thresh=2)
    .reset_index(drop=True)
)

df.set_index("Codigo BDP", inplace=True)
df = df.T
df.index = pd.date_range(start="2016-01-01", periods=len(df),freq='QE')

In [101]:
df

Codigo BDP,B90,A,A.1,A.1.1,A.1.2,A.2,A.2.1,A.2.1.2,A.2.1.4,A.2.2,...,P.4.5.1,P.4.5.2,P.4.5.3,P.4.5.4,P.4.6,P.4.6.1,P.4.6.2,P.4.6.3,P.4.6.4,P.4.7
2016-03-31,36596.746656,277000.273012,38114.429638,38114.429638,0.0,41637.277318,27305.371721,71.833394,27233.538327,14331.905597,...,0.0,124.578439,0.0,17339.505456,2713.813957,0.0,187.683146,0.0,148.829055,2377.301757
2016-06-30,37751.106424,281438.410239,39238.503960,39238.503960,0.0,42402.225402,27703.244723,74.635858,27628.608865,14698.980679,...,0.0,32.206796,0.0,16032.753005,2605.895721,0.0,84.270667,0.0,161.163967,2360.461088
2016-09-30,37369.174947,283617.758686,39444.781982,39444.781982,0.0,42633.423608,28209.017322,76.859853,28132.157468,14424.406286,...,0.0,32.436518,0.0,15211.224340,2851.393995,0.0,254.166744,0.0,241.879112,2355.348139
2016-12-31,44144.009497,291753.609994,39735.115679,39735.115679,0.0,44031.205022,29653.775857,78.415099,29575.360758,14377.429165,...,0.0,24.312220,0.0,15061.046044,2482.432093,0.0,137.152393,0.0,76.803484,2268.476215
2017-03-31,36156.878034,308510.638817,40048.802800,40048.802800,0.0,48048.412335,31415.114588,76.355758,31338.758830,16633.297747,...,0.0,26.157889,0.0,15022.604760,2558.312484,0.0,197.833179,0.0,70.876341,2289.602964
2017-06-30,32877.453316,311568.943100,40343.081308,40343.081308,0.0,51212.286888,32721.042802,75.914992,32645.127810,18491.244086,...,0.0,27.537726,0.0,16069.254230,2706.968518,0.0,280.354701,0.0,78.726675,2347.887142
2017-09-30,22408.100781,322485.207884,40658.133029,40658.133029,0.0,54200.290408,34783.505047,78.188044,34705.317003,19416.785362,...,0.0,71.032054,0.0,16464.364355,2831.166023,0.0,247.611061,0.0,198.703231,2384.851731
2017-12-31,16449.977905,337123.185545,40930.019418,40930.019418,0.0,59404.983427,38805.643747,75.042505,38730.601242,20599.339680,...,0.0,14.762760,0.0,18805.830576,2930.759229,0.0,266.294010,0.0,261.324149,2403.141070
2018-03-31,15031.863406,351948.476936,41298.532178,41298.532178,0.0,62724.107122,37699.636486,0.000000,37699.636486,25024.470636,...,0.0,23.682713,0.0,20173.971554,3156.036766,0.0,415.686506,0.0,280.788339,2459.561921
2018-06-30,35943.559960,360808.645157,41642.847789,41642.847789,0.0,60781.146910,35016.901921,0.000000,35016.901921,25764.244989,...,0.0,25.703025,0.0,22741.315081,2933.772102,0.0,229.351390,0.0,330.918232,2373.502481
